In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import kagglehub
# import statsmodels.api as sm

# import tensorflow_addons as tfa
import cv2
from keras import backend as K

from sklearn.preprocessing import StandardScaler
import math
from sklearn.datasets import load_iris
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Input, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten, AveragePooling2D, Activation, Concatenate
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score
from keras.datasets import mnist
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import SparseCategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import GroupShuffleSplit


import logging, os

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')

import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from scipy.io import wavfile
from scipy.fftpack import fft
import cv2
import os
import librosa
import librosa.display
from glob import glob
import skimage
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from pydub import AudioSegment
from tqdm import tqdm
import tempfile

from tensorflow.keras import regularizers
tf.debugging.set_log_device_placement(False)

import numpy as np
import os
import re


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
from torchmetrics.classification import MulticlassAUROC, MulticlassAccuracy
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


import tensorflow.keras.backend as K
from tensorflow.keras.metrics import Metric

if os.path.isfile("myData.npz"):
# if False:
    print("✅ 'data.npz' exists.")
    with np.load("myData.npz") as data:
        myData = data['myData']
        myData2 = data['myData2']
        myData3 = data['myData3']
        myY = data['myY']
        myActors = data['myActors']
        myDatasets = data['myDatasets']
        myHasNoise = data['myHasNoise']
        print(np.unique(myY))
        print(np.unique(myActors))
else:

    folder = './savefiles'

    def findFilesFromPattern(pattern):
        pattern = re.compile(pattern + r'_(.*?)_(.*?)_(.*?)_(\d+)_(\d+)\.npy')
        heatmaps_dict = {}

        for filename in os.listdir(folder):
            match = pattern.match(filename)
            if match:
                dataset, actor, emotion, i, j = map(str, match.groups())
                i, j = int(i), int(j)
                filepath = os.path.join(folder, filename)
                data = np.load(filepath)

                heatmaps_dict[f'{dataset}_{actor}_{emotion}_{j // 2}_{j%2}'] = {'data': data, 'dataset': dataset, 'actor': actor, 'emotion':emotion, 'type': j, 'has_noise': (j%4 == 0 or j%4==1)}

        return heatmaps_dict

    mfccwasserstein = findFilesFromPattern('wassersteinMfccHeat')
    melwasserstein = findFilesFromPattern('wassersteinHeat')
    meltimeeuclid = findFilesFromPattern('timeMetricHeat')
    meleuclid = findFilesFromPattern('euclideanHeat')

    def load_spectrograms(prefixes, path='./savefiles'):
        patterns = []
        for prefix in prefixes:
            patterns.append(os.path.join(path, f"{prefix}_*.npy"))
        my_globs = glob(patterns[0])
        for pattern in patterns[1:]:
            my_globs = my_globs + glob(pattern)
        file_list = sorted(my_globs)
        return [np.load(file) for i, file in enumerate(file_list)]

    myRaw = load_spectrograms(["savee", 'tess', 'radvess', 'cremad'])
    print(len(mfccwasserstein))
    print(len([mfccwasserstein[key]['data'] for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 0]))
    print(len([mfccwasserstein[key]['data'] for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 1]))
    print(np.array([[meleuclid[key]['data'] for key in sorted(meleuclid.keys()) if meleuclid[key]['type'] == 0]]).shape)

    myRaw2 = load_spectrograms(["mfcc_savee", 'mfcc_tess', 'mfcc_radvess', 'mfcc_cremad'])

    print(len(myRaw2))
    print(len(myRaw2[0]))

    myData = np.array([myRaw])
    myData3 = np.stack([myRaw2])
    print('finish data')
    myData = myData.astype('float16')
    myData = np.transpose(myData, (1, 2, 3, 0))

    myData3 = myData3.astype('float16')
    myData3 = np.transpose(myData3, (1, 2, 3, 0))
    myEmotionMap = {
        'neutral': 1, 'calm':2, 'happy':3, 'sad':4, 'angry':5, 'fearful':6, 'disgust':7, 'surprised':8
    }
    myY = np.array(
        [myEmotionMap[mfccwasserstein[key]['emotion']] -1 for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 0]
    )
    print(np.unique(myY))
    myActors = np.array(
        [mfccwasserstein[key]['actor'] + '_' + mfccwasserstein[key]['dataset']  for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 0]
    )
    myDatasets = np.array(
        [mfccwasserstein[key]['dataset']  for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 0]
    )
    myHasNoise = np.array(
        [mfccwasserstein[key]['has_noise']  for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 0]
    )
    print(np.unique(myActors))

    print(np.unique(myY))
    myY = [x -1 if x > 0 else x for x in myY]
    myY = to_categorical(myY, num_classes=6)

    myData2 = np.array([
                        [meleuclid[key]['data'] for key in sorted(meleuclid.keys()) if meleuclid[key]['type'] % 2 == 0],
                        [meleuclid[key]['data'] for key in sorted(meleuclid.keys()) if meleuclid[key]['type'] % 2 == 1],
                        [meltimeeuclid[key]['data'] for key in sorted(meltimeeuclid.keys()) if meltimeeuclid[key]['type'] % 2 == 0],
                        [meltimeeuclid[key]['data'] for key in sorted(meltimeeuclid.keys()) if meltimeeuclid[key]['type'] % 2 == 1],
                        # [mfccwasserstein[key]['data'] for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 0],
                        # [mfccwasserstein[key]['data'] for key in sorted(mfccwasserstein.keys()) if mfccwasserstein[key]['type'] % 2 == 1],
                        [melwasserstein[key]['data'] for key in sorted(melwasserstein.keys()) if melwasserstein[key]['type'] % 2 == 0],
                        [melwasserstein[key]['data'] for key in sorted(melwasserstein.keys()) if melwasserstein[key]['type'] % 2 == 1]
                        ])
    print('finish data')
    myData2 = myData2.astype('float16')
    print(myData2.shape)
    myData2 = np.transpose(myData2, (1, 2, 3, 0))
    print(myData2.shape)

    def upload_to_gcs(bucket_name, source_file, destination_blob):
        """Upload a file to a GCS bucket."""
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(destination_blob)
        blob.upload_from_filename(source_file)
        print(f"✅ Uploaded {destination_blob} to gs://{bucket_name}/{destination_blob}")

    np.savez_compressed(
        "myData.npz",
        myData=myData,
        myData2=myData2,
        myData3=myData3,
        myY=myY,
        myActors=myActors,
        myDatasets=myDatasets,
        myHasNoise=myHasNoise
    )

    GCS_BUCKET = "simplicialcomplex-outputbucket"

    # upload_to_gcs(GCS_BUCKET, "myData.npz", "data/myData.npz")


In [ ]:
# np.unique([str(x.shape) for x in myRaw2])

In [ ]:
splitter = GroupShuffleSplit(test_size=0.2, n_splits=1)
groups = myActors#np.array([f"{d}_{a}_{c}" for d, a, c in zip(myActors, myDatasets)])

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np

def stratified_group_split_3way(y, z, groups, val_size=0.2, test_size=0.2, random_state=42):
    rng = np.random.default_rng(random_state)
    df = pd.DataFrame({'y': y, 'z': z, 'group': groups})

    df["strat_label"] = df["y"].astype(str) + "_" + df["z"].astype(str)

    group_labels = (
        df.groupby('group')['y']
          .agg(lambda s: s.value_counts().index[0])
          .reset_index()
    )

    sss_outer = StratifiedShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state
    )

    sss_inner = StratifiedShuffleSplit(
        n_splits=1, test_size=val_size/(1 - test_size), random_state=random_state + 1
    )

    group_indices = np.arange(len(group_labels))
    for trainval_g, test_g in sss_outer.split(group_indices, group_labels['y']):
        trainval_groups = group_labels['group'].iloc[trainval_g].values
        test_groups = group_labels['group'].iloc[test_g].values

        trainval_df = group_labels.iloc[trainval_g]
        trainval_idx = np.arange(len(trainval_df))

        for train_g, val_g in sss_inner.split(trainval_idx, trainval_df['y']):
            train_groups = trainval_df['group'].iloc[train_g].values
            val_groups = trainval_df['group'].iloc[val_g].values

    # Map back to samples
    train_mask = df['group'].isin(train_groups)
    val_mask = df['group'].isin(val_groups)
    test_mask = df['group'].isin(test_groups)

    train_idx = np.where(train_mask)[0]
    val_idx = np.where(val_mask)[0]
    test_idx = np.where(test_mask)[0]

    return train_idx, val_idx, test_idx

def stratified_group_split_cover_z_3way(
    y,
    z,
    groups,
    val_size=0.2,
    test_size=0.2,
    random_state=42,
    keep_size=True,
):
    """
    3-way group-wise split (train/val/test) with:
      - stratification by composite (y,z)
      - at least one of each z in test
      - no group overlap between splits

    Parameters
    ----------
    y : array-like
        Primary labels (e.g. emotion IDs)
    z : array-like
        Secondary variable (e.g. gender, dataset)
    groups : array-like
        Group IDs (e.g. actors)
    val_size : float
        Fraction of remaining groups for validation (after test split)
    test_size : float
        Fraction of all groups for test
    random_state : int
        RNG seed
    keep_size : bool
        Try to maintain split sizes even when adding missing z-groups
    """
    rng = np.random.default_rng(random_state)
    df = pd.DataFrame({"y": y, "z": z, "group": groups})

    # Composite stratification label
    df["strat"] = df["y"].astype(str) + "_" + df["z"].astype(str)

    # Majority composite per group
    grp = (
        df.groupby("group")["y"]
          .agg(lambda s: s.value_counts().index[0])
          .reset_index()
    )

    # --- Step 1: TrainVal/Test Split ---
    sss_outer = StratifiedShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state
    )
    g_idx = np.arange(len(grp))
    for trainval_g_idx, test_g_idx in sss_outer.split(g_idx, grp["y"]):
        trainval_groups = set(grp["group"].iloc[trainval_g_idx])
        test_groups = set(grp["group"].iloc[test_g_idx])

    # --- Ensure all z categories appear in test ---
    all_z = set(df["z"].unique())
    def z_in_groups(gset):
        return set(df[df["group"].isin(gset)]["z"].unique())

    test_z = z_in_groups(test_groups)
    missing_z = list(all_z - test_z)

    if missing_z:
        g2count = df.groupby("group").size().to_dict()
        z_to_groups = {zv: set(df.loc[df["z"] == zv, "group"].unique()) for zv in all_z}

        for zv in missing_z:
            candidates = z_to_groups[zv] & trainval_groups
            if not candidates:
                continue
            g_chosen = min(candidates, key=lambda g: g2count.get(g, 0))
            trainval_groups.remove(g_chosen)
            test_groups.add(g_chosen)

            if keep_size:
                removable = []
                test_z_after = z_in_groups(test_groups)
                for g in list(test_groups - {g_chosen}):
                    tmp = test_groups.copy()
                    tmp.remove(g)
                    if z_in_groups(tmp) == test_z_after:
                        removable.append(g)
                if removable:
                    g_back = max(removable, key=lambda g: g2count.get(g, 0))
                    test_groups.remove(g_back)
                    trainval_groups.add(g_back)

    # --- Step 2: Train/Val Split (from remaining groups) ---
    subgrp = (
        df[df["group"].isin(trainval_groups)]
        .groupby("group")["y"]
        .agg(lambda s: s.value_counts().index[0])
        .reset_index()
    )

    sss_inner = StratifiedShuffleSplit(
        n_splits=1,
        test_size=val_size / (1 - test_size),
        random_state=random_state + 1,
    )
    g_idx_inner = np.arange(len(subgrp))
    for train_g_idx, val_g_idx in sss_inner.split(g_idx_inner, subgrp["y"]):
        train_groups = set(subgrp["group"].iloc[train_g_idx])
        val_groups = set(subgrp["group"].iloc[val_g_idx])

    # --- Map groups to sample indices ---
    train_idx = np.where(df["group"].isin(train_groups))[0]
    val_idx   = np.where(df["group"].isin(val_groups))[0]
    test_idx  = np.where(df["group"].isin(test_groups))[0]

    return train_idx, val_idx, test_idx




train_idx, val_idx, test_idx = stratified_group_split_cover_z_3way(y=np.argmax(myY, axis=1), z=myDatasets, groups=groups, val_size=0.2, test_size=0.2)

X_train, X_val, X_test, X_train2, X_val2, X_test2, X_train3, X_val3, X_test3 = myData[train_idx], myData[val_idx], myData[test_idx], myData2[train_idx], myData2[val_idx], myData2[test_idx], myData3[train_idx], myData3[val_idx], myData3[test_idx]
y_train, y_val, y_test = myY[train_idx], myY[val_idx], myY[test_idx]


print(X_train.shape)
has_noise_idx = np.where(myHasNoise[train_idx] == 0)[0]
X_train_tensor = torch.tensor(X_train[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
X_train2_tensor = torch.tensor(X_train2[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
X_train3_tensor = torch.tensor(X_train3[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
y_train_tensor = torch.tensor(np.argmax(y_train[has_noise_idx], axis=1), dtype=torch.long)

has_noise_idx = np.where(myHasNoise[val_idx] == 0)[0]
X_val_tensor = torch.tensor(X_val[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
X_val2_tensor = torch.tensor(X_val2[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
X_val3_tensor = torch.tensor(X_val3[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
y_val_tensor = torch.tensor(np.argmax(y_val[has_noise_idx], axis=1), dtype=torch.long)

has_noise_idx = np.where(myHasNoise[test_idx] == 0)[0]
X_test_tensor = torch.tensor(X_test[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
X_test2_tensor = torch.tensor(X_test2[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
X_test3_tensor = torch.tensor(X_test3[has_noise_idx].transpose(0, 3, 1, 2), dtype=torch.float32)
y_test_tensor = torch.tensor(np.argmax(y_test[has_noise_idx], axis=1), dtype=torch.long)


In [ ]:


class CNNModel(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),  # for input 32×32 after two poolings
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

class CNNModel2(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNModel2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(6, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

class CombinedModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = CNNModel(64)
        self.model2 = CNNModel2(64)
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
    def forward(self, x1, x2):
        x1 = self.model(x1)
        x2 = self.model2(x2)
        x = torch.cat([x1, x2], dim=1)
        return self.classifier(x)

if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple GPU
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU (if any)
else:
    device = torch.device("cpu")   # fallback

auroc = MulticlassAUROC(num_classes=6, average='macro').to(device)
top3acc = MulticlassAccuracy(num_classes=6, top_k=3).to(device)

In [ ]:
np.unique(myActors[train_idx])

In [ ]:
np.unique(myActors[test_idx])

In [ ]:
np.unique(myActors[val_idx])

In [ ]:
np.unique(y_train)

In [ ]:

# --- Instantiate model ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


dataset = TensorDataset(X_train_tensor, X_train2_tensor, y_train_tensor)
val_ds = TensorDataset(X_val_tensor, X_val2_tensor, y_val_tensor)

train_loader = DataLoader(dataset, batch_size=256, shuffle=True,
                          num_workers=4,
    pin_memory=True,
    prefetch_factor=4,
    persistent_workers=True)
val_loader = DataLoader(val_ds, batch_size=256,num_workers=4,
    pin_memory=True,
    prefetch_factor=4,
    persistent_workers=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, X_test2_tensor, y_test_tensor), batch_size=256)



# model = CNNModel().to(device)
# model2 = CNNModel2().to(device)


# fusion = FusionNetGated().to(device)

model_comb = CombinedModel(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
# optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3, weight_decay=1e-4)
# fusion_optimizer = torch.optim.Adam(fusion.parameters(), lr=1e-3, weight_decay=1e-4)
optimizer_combined = torch.optim.Adam(model_comb.parameters())
num_epochs = 30

best_val_auc = 0


# ================================================================
# Training Loop with Checkpoint
# ================================================================
for epoch in range(num_epochs):
    model_comb.train()
    train_preds, train_labels = [], []
    for X_batch, X2_batch, y_batch in train_loader:
        X_batch, X2_batch, y_batch = X_batch.to(device), X2_batch.to(device), y_batch.to(device)
        # optimizer.zero_grad()
        # optimizer2.zero_grad()
        # fusion_optimizer.zero_grad()
        optimizer_combined.zero_grad()

        # outputs = model(X_batch)
        # outputs2 = model2(X2_batch)

        # combined_logits, alpha = fusion(outputs, outputs2)
        combined_logits = model_comb(X_batch, X2_batch)
        loss = criterion(combined_logits, y_batch)

        # optional deep supervision helps
        # loss = loss + 0.2*criterion(outputs, y_batch) + 0.2*criterion(outputs2, y_batch)
        loss.backward()

        train_preds.append(torch.softmax(combined_logits, dim=1))
        train_labels.append(y_batch)

        # optimizer.step()

        # optimizer2.step()

        # fusion_optimizer.step()

        optimizer_combined.step()

    # Validation
    model_comb.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for X_val, X2_val, y_val in val_loader:
            X_val, X2_val, y_val = X_val.to(device), X2_val.to(device), y_val.to(device)
            # outputs = model(X_val)
            # outputs2 = model2(X2_val)

            # combined_logits = fusion(outputs, outputs2)
            # combined_logits, alpha = fusion(outputs, outputs2)

            combined_logits = model_comb(X_val, X2_val)
            # loss = criterion(combined_logits, y_val)

            val_loss = criterion(combined_logits, y_val)
            preds = torch.softmax(combined_logits, dim=1)
            val_preds.append(preds)
            val_labels.append(y_val)
    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)

    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f} - val_loss {val_loss.item():.4f}")

    # Save best model
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        # torch.save(model.state_dict(), "best_model_full.pth")
        # torch.save(model2.state_dict(), "best_model_full2.pth")
        # torch.save(fusion.state_dict(), "best_model_full3.pth")
        torch.save(model_comb.state_dict(), "best_model_comb.pth")
        print("✅ Saved new best model.")

    train_preds = torch.cat(train_preds)
    train_labels = torch.cat(train_labels)
    val_auc = auroc(train_preds, train_labels).item()
    val_top3 = top3acc(train_preds, train_labels).item()

    y_pred = torch.argmax(train_preds, dim=1)

    accuracy = (y_pred == train_labels).float().mean()
    print(f"Epoch {epoch+1}/{num_epochs} - train_auc: {val_auc:.4f} - train_top3_acc: {val_top3:.4f} - train_acc: {accuracy.item():.4f} - train_loss: {loss.item():.4f}")


In [ ]:

# ================================================================
# Evaluation
# ================================================================
# model.load_state_dict(torch.load("best_model_full.pth"))
# model2.load_state_dict(torch.load("best_model_full2.pth"))
# fusion.load_state_dict(torch.load("best_model_full3.pth"))
# model.eval()
# model2.eval()
# fusion.eval()
model_comb.load_state_dict(torch.load("best_model_comb.pth"))
model_comb.eval()

all_preds, all_labels, all_preds2, all_labels2 = [], [], [], []
with torch.no_grad():
    for X_batch, X2_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        X2_batch = X2_batch.to(device)
        # outputs = model(X_batch)
        # outputs2 = model2(X2_batch)

        # combined_logits = fusion(outputs, outputs2)
        # combined_logits = fusion(outputs, outputs2)
        combined_logits = model_comb(X_batch, X2_batch)
        # preds = torch.softmax(combined_logits, dim=1)
        preds = torch.argmax(combined_logits, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_preds2.append(torch.softmax(combined_logits, dim=1))
        all_labels.extend(y_batch.numpy())
        all_labels2.append(y_batch.to(device))

    val_preds = torch.cat(all_preds2)
    val_labels = torch.cat(all_labels2)
    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f}")

# Classification report
class_labels = ['neutral', 'happy', 'sad', 'angry', 'fearful', 'disgust']
report = classification_report(all_labels, all_preds, target_names=class_labels)
print(report)

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),  # for input 32×32 after two poolings
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 64),  # for input 32×32 after two poolings
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:

dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_ds = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=256)

model = CNNModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 20

best_val_auc = 0.0

# ================================================================
# Training Loop with Checkpoint
# ================================================================
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            outputs = model(X_val)
            preds = torch.softmax(outputs, dim=1)
            val_preds.append(preds)
            val_labels.append(y_val)
    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)

    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f}")

    # Save best model
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), "best_model_mel.pth")
        print("✅ Saved new best model.")


In [ ]:

# ================================================================
# Evaluation
# ================================================================
model.load_state_dict(torch.load("best_model_mel.pth"))
model.eval()

all_preds, all_labels, all_preds2, all_labels2 = [], [], [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_preds2.append(torch.softmax(outputs, dim=1))
        all_labels.extend(y_batch.numpy())
        all_labels2.append(y_batch.to(device))

    val_preds = torch.cat(all_preds2)
    val_labels = torch.cat(all_labels2)
    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f}")


# Classification report
class_labels = ['neutral', 'happy', 'sad', 'angry', 'fearful', 'disgust']
report = classification_report(all_labels, all_preds, target_names=class_labels)
print(report)

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
class CNNModel2(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNModel2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(6, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:

dataset = TensorDataset(X_train2_tensor, y_train_tensor)
val_ds = TensorDataset(X_val2_tensor, y_val_tensor)

train_loader = DataLoader(dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256)
test_loader = DataLoader(TensorDataset(X_test2_tensor, y_test_tensor), batch_size=256)

model = CNNModel2().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 40

best_val_auc = 0.0

# ================================================================
# Training Loop with Checkpoint
# ================================================================
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            outputs = model(X_val)
            preds = torch.softmax(outputs, dim=1)
            val_preds.append(preds)
            val_labels.append(y_val)
    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)

    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f}")

    # Save best model
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), "best_model_per.pth")
        print("✅ Saved new best model.")


In [ ]:

# ================================================================
# Evaluation
# ================================================================
model.load_state_dict(torch.load("best_model_per.pth"))
model.eval()

all_preds, all_labels, all_preds2, all_labels2 = [], [], [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_preds2.append(torch.softmax(outputs, dim=1))
        all_labels.extend(y_batch.numpy())
        all_labels2.append(y_batch.to(device))

    val_preds = torch.cat(all_preds2)
    val_labels = torch.cat(all_labels2)
    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f}")


# Classification report
class_labels = ['neutral', 'happy', 'sad', 'angry', 'fearful', 'disgust']
report = classification_report(all_labels, all_preds, target_names=class_labels)
print(report)

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
class CNNModel3(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNModel3, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:

dataset = TensorDataset(X_train3_tensor, y_train_tensor)
val_ds = TensorDataset(X_val3_tensor, y_val_tensor)

train_loader = DataLoader(dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256)
test_loader = DataLoader(TensorDataset(X_test3_tensor, y_test_tensor), batch_size=256)

model = CNNModel3().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 50

best_val_auc = 0.0

# ================================================================
# Training Loop with Checkpoint
# ================================================================
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            outputs = model(X_val)
            preds = torch.softmax(outputs, dim=1)
            val_preds.append(preds)
            val_labels.append(y_val)
    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)

    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f}")

    # Save best model
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), "best_model_per.pth")
        print("✅ Saved new best model.")


In [ ]:

# ================================================================
# Evaluation
# ================================================================
model.load_state_dict(torch.load("best_model_per.pth"))
model.eval()

all_preds, all_labels, all_preds2, all_labels2 = [], [], [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_preds2.append(torch.softmax(outputs, dim=1))
        all_labels.extend(y_batch.numpy())
        all_labels2.append(y_batch.to(device))

    val_preds = torch.cat(all_preds2)
    val_labels = torch.cat(all_labels2)
    val_auc = auroc(val_preds, val_labels).item()
    val_top3 = top3acc(val_preds, val_labels).item()

    y_pred = torch.argmax(val_preds, dim=1)

    accuracy = (y_pred == val_labels).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs} - val_auc: {val_auc:.4f} - top3_acc: {val_top3:.4f} - val_acc: {accuracy.item():.4f}")


# Classification report
class_labels = ['neutral', 'happy', 'sad', 'angry', 'fearful', 'disgust']
report = classification_report(all_labels, all_preds, target_names=class_labels)
print(report)

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()